In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv(r'DOH COVID Data Drop_ 20201023 - 04 Case Information.csv')
data

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,C281814,74.0,70 to 74,MALE,2020-07-25,2020-07-27,2020-07-31,NaN,NaN,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
1,C592217,33.0,30 to 34,MALE,2020-07-16,2020-07-21,2020-07-25,NaN,2020-08-06,RECOVERED,...,NCR,NaN,NaN,NaN,NaN,RECOVERED,YES,NaN,NaN,NaN
2,C865185,30.0,30 to 34,FEMALE,2020-07-17,2020-07-21,2020-07-25,NaN,NaN,RECOVERED,...,NCR,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NO,"Health Status is ""Recovered"", but no Date Reco..."
3,C460178,4.0,0 to 4,FEMALE,2020-07-20,2020-07-20,2020-07-30,NaN,NaN,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NO,"Health Status is ""Recovered"", but no Date Reco..."
4,C898472,29.0,25 to 29,FEMALE,NaN,2020-04-22,2020-04-23,NaN,2020-05-10,RECOVERED,...,NCR,CALOOCAN CITY,PH137501000,NaN,NaN,RECOVERED,NO,2020-04-15,NO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365794,C881229,29.0,25 to 29,MALE,2020-09-09,2020-09-13,2020-09-16,NaN,2020-09-24,RECOVERED,...,ROMBLON,CALATRAVA,PH175904000,BALOGO,PH175904001,RECOVERED,YES,NaN,NaN,NaN
365795,C217601,26.0,25 to 29,FEMALE,2020-09-09,2020-09-13,2020-09-16,NaN,2020-09-24,RECOVERED,...,ROMBLON,CALATRAVA,PH175904000,BALOGO,PH175904001,RECOVERED,YES,NaN,NO,NaN
365796,C955729,35.0,35 to 39,MALE,2020-10-08,2020-10-11,2020-10-14,NaN,NaN,NaN,...,ROMBLON,SAN FERNANDO,PH175913000,POBLACION,PH175913011,MILD,YES,2020-10-05,NaN,NaN
365797,C768847,46.0,45 to 49,FEMALE,2020-08-24,2020-08-28,2020-08-30,NaN,2020-09-07,RECOVERED,...,ROMBLON,SANTA MARIA (IMELDA),PH175917000,NaN,NaN,RECOVERED,YES,2020-08-20,NO,NaN


In [3]:
data.corr()

,Age
Age,1.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365799 entries, 0 to 365798
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CaseCode           365799 non-null  object 
 1   Age                363606 non-null  float64
 2   AgeGroup           363606 non-null  object 
 3   Sex                365799 non-null  object 
 4   DateSpecimen       309198 non-null  object 
 5   DateResultRelease  310344 non-null  object 
 6   DateRepConf        365799 non-null  object 
 7   DateDied           6880 non-null    object 
 8   DateRecover        84903 non-null   object 
 9   RemovalType        319606 non-null  object 
 10  Admitted           365648 non-null  object 
 11  RegionRes          362100 non-null  object 
 12  ProvRes            340886 non-null  object 
 13  CityMunRes         330338 non-null  object 
 14  CityMuniPSGC       330338 non-null  object 
 15  BarangayRes        137015 non-null  object 
 16  Ba

In [5]:
df = data.drop('CaseCode', axis=1)
df = df.drop(df.loc[:,'AgeGroup':'Sex'].columns, axis=1)
df = df.drop(df.loc[:, 'DateResultRelease':'Admitted'].columns, axis=1)
df = df.drop('ProvRes', axis=1)
df = df.drop(df.loc[:, 'CityMuniPSGC':'BarangayPSGC'].columns, axis=1)
df = df.drop(df.loc[:, 'DateOnset':'ValidationStatus'].columns, axis=1)
df

,Age,DateSpecimen,RegionRes,CityMunRes,HealthStatus,Quarantined
0,74.0,2020-07-25,NCR,NaN,RECOVERED,NO
1,33.0,2020-07-16,NCR,NaN,RECOVERED,YES
2,30.0,2020-07-17,NCR,NaN,RECOVERED,NO
3,4.0,2020-07-20,NCR,NaN,RECOVERED,NO
4,29.0,NaN,NCR,CALOOCAN CITY,RECOVERED,NO
...,...,...,...,...,...,...
365794,29.0,2020-09-09,Region IV-B: MIMAROPA,CALATRAVA,RECOVERED,YES
365795,26.0,2020-09-09,Region IV-B: MIMAROPA,CALATRAVA,RECOVERED,YES
365796,35.0,2020-10-08,Region IV-B: MIMAROPA,SAN FERNANDO,MILD,YES
365797,46.0,2020-08-24,Region IV-B: MIMAROPA,SANTA MARIA (IMELDA),RECOVERED,YES


In [6]:
titles = list(df.columns)
titles

['Age',
 'DateSpecimen',
 'RegionRes',
 'CityMunRes',
 'HealthStatus',
 'Quarantined']

In [7]:
titles[4], titles[5] = titles[5], titles[4]
titles[2], titles[3] = titles[3], titles[2]
titles

['Age',
 'DateSpecimen',
 'CityMunRes',
 'RegionRes',
 'Quarantined',
 'HealthStatus']

In [8]:
df = df[titles]
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,74.0,2020-07-25,NaN,NCR,NO,RECOVERED
1,33.0,2020-07-16,NaN,NCR,YES,RECOVERED
2,30.0,2020-07-17,NaN,NCR,NO,RECOVERED
3,4.0,2020-07-20,NaN,NCR,NO,RECOVERED
4,29.0,NaN,CALOOCAN CITY,NCR,NO,RECOVERED
...,...,...,...,...,...,...
365794,29.0,2020-09-09,CALATRAVA,Region IV-B: MIMAROPA,YES,RECOVERED
365795,26.0,2020-09-09,CALATRAVA,Region IV-B: MIMAROPA,YES,RECOVERED
365796,35.0,2020-10-08,SAN FERNANDO,Region IV-B: MIMAROPA,YES,MILD
365797,46.0,2020-08-24,SANTA MARIA (IMELDA),Region IV-B: MIMAROPA,YES,RECOVERED


In [9]:
start_date = '2020-09-10'
end_date = '2020-10-10'

df = df.loc[df['RegionRes'] == 'NCR']
df = df.loc[(df['DateSpecimen'] >= start_date) & ((df['DateSpecimen'] <= end_date))]
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
198,24.0,2020-09-24,CITY OF NAVOTAS,NCR,YES,RECOVERED
402,26.0,2020-10-02,CITY OF MANILA,NCR,NO,MILD
993,88.0,2020-09-11,CITY OF SAN JUAN,NCR,NO,RECOVERED
1190,85.0,2020-10-07,CALOOCAN CITY,NCR,NO,DIED
1828,46.0,2020-09-21,CITY OF NAVOTAS,NCR,YES,RECOVERED
...,...,...,...,...,...,...
361617,56.0,2020-09-22,TAGUIG CITY,NCR,NO,RECOVERED
361632,57.0,2020-09-28,TAGUIG CITY,NCR,NO,RECOVERED
361691,41.0,2020-09-23,TAGUIG CITY,NCR,NO,RECOVERED
361696,33.0,2020-10-02,TAGUIG CITY,NCR,NO,MILD


In [10]:
cities = df['CityMunRes'].unique().tolist()
#df = df.loc[(df['CityMunRes'] == 'CITY OF ANTIPOLO (CAPITAL)') | (df['CityMunRes'] == 'TAYTAY')]
#df = df.reset_index(drop=True)

cities

['CITY OF NAVOTAS',
 'CITY OF MANILA',
 'CITY OF SAN JUAN',
 'CALOOCAN CITY',
 'QUEZON CITY',
 'CITY OF MARIKINA',
 nan,
 'CITY OF MALABON',
 'CITY OF MAKATI',
 'CITY OF PASIG',
 'CITY OF MUNTINLUPA',
 'PASAY CITY',
 'CITY OF PARAÑAQUE',
 'TAGUIG CITY',
 'CITY OF MANDALUYONG',
 'CITY OF VALENZUELA',
 'CITY OF LAS PIÑAS',
 'PATEROS']

In [11]:
df = df.loc[(df['CityMunRes'] == 'QUEZON CITY') | (df['CityMunRes'] == 'CITY OF MAKATI')]
df = df.reset_index(drop=True)
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,22.0,2020-10-01,QUEZON CITY,NCR,NO,MILD
1,28.0,2020-09-24,QUEZON CITY,NCR,NO,RECOVERED
2,27.0,2020-10-08,QUEZON CITY,NCR,NO,MILD
3,59.0,2020-09-16,QUEZON CITY,NCR,NO,RECOVERED
4,25.0,2020-09-30,QUEZON CITY,NCR,NO,MILD
...,...,...,...,...,...,...
6015,25.0,2020-09-18,CITY OF MAKATI,NCR,NO,RECOVERED
6016,76.0,2020-09-15,CITY OF MAKATI,NCR,YES,RECOVERED
6017,9.0,2020-09-20,CITY OF MAKATI,NCR,YES,RECOVERED
6018,45.0,2020-09-19,CITY OF MAKATI,NCR,NO,RECOVERED


In [12]:
df = df.dropna()
df = df.reset_index(drop=True)
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,22.0,2020-10-01,QUEZON CITY,NCR,NO,MILD
1,28.0,2020-09-24,QUEZON CITY,NCR,NO,RECOVERED
2,27.0,2020-10-08,QUEZON CITY,NCR,NO,MILD
3,59.0,2020-09-16,QUEZON CITY,NCR,NO,RECOVERED
4,25.0,2020-09-30,QUEZON CITY,NCR,NO,MILD
...,...,...,...,...,...,...
6015,25.0,2020-09-18,CITY OF MAKATI,NCR,NO,RECOVERED
6016,76.0,2020-09-15,CITY OF MAKATI,NCR,YES,RECOVERED
6017,9.0,2020-09-20,CITY OF MAKATI,NCR,YES,RECOVERED
6018,45.0,2020-09-19,CITY OF MAKATI,NCR,NO,RECOVERED


In [13]:
df['DateSpecimen'] = pd.to_datetime(df['DateSpecimen'], format='%Y-%m-%d')
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,22.0,2020-10-01,QUEZON CITY,NCR,NO,MILD
1,28.0,2020-09-24,QUEZON CITY,NCR,NO,RECOVERED
2,27.0,2020-10-08,QUEZON CITY,NCR,NO,MILD
3,59.0,2020-09-16,QUEZON CITY,NCR,NO,RECOVERED
4,25.0,2020-09-30,QUEZON CITY,NCR,NO,MILD
...,...,...,...,...,...,...
6015,25.0,2020-09-18,CITY OF MAKATI,NCR,NO,RECOVERED
6016,76.0,2020-09-15,CITY OF MAKATI,NCR,YES,RECOVERED
6017,9.0,2020-09-20,CITY OF MAKATI,NCR,YES,RECOVERED
6018,45.0,2020-09-19,CITY OF MAKATI,NCR,NO,RECOVERED


In [14]:
df['DateSpecimen'] = df['DateSpecimen'].dt.strftime('%m/%d/%Y')
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,22.0,10/01/2020,QUEZON CITY,NCR,NO,MILD
1,28.0,09/24/2020,QUEZON CITY,NCR,NO,RECOVERED
2,27.0,10/08/2020,QUEZON CITY,NCR,NO,MILD
3,59.0,09/16/2020,QUEZON CITY,NCR,NO,RECOVERED
4,25.0,09/30/2020,QUEZON CITY,NCR,NO,MILD
...,...,...,...,...,...,...
6015,25.0,09/18/2020,CITY OF MAKATI,NCR,NO,RECOVERED
6016,76.0,09/15/2020,CITY OF MAKATI,NCR,YES,RECOVERED
6017,9.0,09/20/2020,CITY OF MAKATI,NCR,YES,RECOVERED
6018,45.0,09/19/2020,CITY OF MAKATI,NCR,NO,RECOVERED


In [15]:
qc = df.loc[df['CityMunRes'] == 'QUEZON CITY']
mkt = df.loc[df['CityMunRes'] == 'CITY OF MAKATI']

qc = qc.reset_index(drop=True)
mkt = mkt.reset_index(drop=True)

In [16]:
qc

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,22.0,10/01/2020,QUEZON CITY,NCR,NO,MILD
1,28.0,09/24/2020,QUEZON CITY,NCR,NO,RECOVERED
2,27.0,10/08/2020,QUEZON CITY,NCR,NO,MILD
3,59.0,09/16/2020,QUEZON CITY,NCR,NO,RECOVERED
4,25.0,09/30/2020,QUEZON CITY,NCR,NO,MILD
...,...,...,...,...,...,...
4462,54.0,09/14/2020,QUEZON CITY,NCR,NO,RECOVERED
4463,25.0,09/14/2020,QUEZON CITY,NCR,NO,RECOVERED
4464,36.0,09/21/2020,QUEZON CITY,NCR,NO,RECOVERED
4465,59.0,09/25/2020,QUEZON CITY,NCR,NO,RECOVERED


In [17]:
mkt

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,44.0,09/14/2020,CITY OF MAKATI,NCR,NO,RECOVERED
1,35.0,09/23/2020,CITY OF MAKATI,NCR,NO,RECOVERED
2,78.0,10/08/2020,CITY OF MAKATI,NCR,NO,ASYMPTOMATIC
3,57.0,09/29/2020,CITY OF MAKATI,NCR,YES,SEVERE
4,51.0,09/24/2020,CITY OF MAKATI,NCR,YES,SEVERE
...,...,...,...,...,...,...
1548,25.0,09/18/2020,CITY OF MAKATI,NCR,NO,RECOVERED
1549,76.0,09/15/2020,CITY OF MAKATI,NCR,YES,RECOVERED
1550,9.0,09/20/2020,CITY OF MAKATI,NCR,YES,RECOVERED
1551,45.0,09/19/2020,CITY OF MAKATI,NCR,NO,RECOVERED


In [18]:
#1st City = QC
qcHealth = qc['HealthStatus'].unique().tolist()
qcDate = qc['DateSpecimen'].unique().tolist()
qcQuar = qc['Quarantined'].unique().tolist()

#2nd City = MKT
mktHealth = mkt['HealthStatus'].unique().tolist()
mktDate = mkt['DateSpecimen'].unique().tolist()
mktQuar = mkt['Quarantined'].unique().tolist()

#Overall
uniHealth = df['HealthStatus'].unique().tolist()
uniDate = df['DateSpecimen'].unique().tolist()
uniQuar = df['Quarantined'].unique().tolist()

In [19]:
#1st City = QC
for i in range(len(qc)):
    for l in qcHealth:
        if qc['HealthStatus'][i] == l:
            qc['HealthStatus'][i] = qcHealth.index(l)
            
    for m in qcDate:
        if qc['DateSpecimen'][i] == m:
            qc['DateSpecimen'][i] = qcDate.index(m)
    
    for n in qcQuar:
        if qc['Quarantined'][i] == n:
            qc['Quarantined'][i] = qcQuar.index(n)
   
#2nd City = Mkt
for i in range(len(mkt)):
    for l in mktHealth:
        if mkt['HealthStatus'][i] == l:
            mkt['HealthStatus'][i] = mktHealth.index(l)
            
    for m in uniDate:
        if mkt['DateSpecimen'][i] == m:
            mkt['DateSpecimen'][i] = mktDate.index(m)
    
    for n in mktQuar:
        if mkt['Quarantined'][i] == n:
            mkt['Quarantined'][i] = mktQuar.index(n)
            
#Overall
for i in range(len(df)):
    for l in uniHealth:
        if df['HealthStatus'][i] == l:
            df['HealthStatus'][i] = uniHealth.index(l)
            
    for m in uniDate:
        if df['DateSpecimen'][i] == m:
            df['DateSpecimen'][i] = uniDate.index(m)

    for n in uniQuar:
        if df['Quarantined'][i] == n:
            df['Quarantined'][i] = uniQuar.index(n)

In [23]:
qc[['HealthStatus', 'DateSpecimen', 'Quarantined']] = qc[['HealthStatus', 'DateSpecimen', 'Quarantined']].apply(pd.to_numeric)
mkt[['HealthStatus', 'DateSpecimen', 'Quarantined']] = mkt[['HealthStatus', 'DateSpecimen', 'Quarantined']].apply(pd.to_numeric)

In [27]:
mkt.corr()

,Age,DateSpecimen,Quarantined,HealthStatus
Age,1.000000,-0.040689,0.137805,-0.017609
DateSpecimen,-0.040689,1.000000,-0.036559,0.487211
Quarantined,0.137805,-0.036559,1.000000,-0.112097
HealthStatus,-0.017609,0.487211,-0.112097,1.000000


In [ ]:
mkt.corr(method='pearson')

In [ ]:
y2.corr(z2, method='pearson')

In [ ]:
yy.corr(zz, method='pearson')

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(y, z, test_size = 0.2)
#x_train2, x_test2, y_train2, y_test2 = train_test_split(y2, z2, test_size = 0.2)

In [ ]:
#logreg = LogisticRegression()
#logreg2 = LogisticRegression()
clf = DecisionTreeClassifier()

In [ ]:
#logreg.fit(x_train, y_train)
#logreg2.fit(x_train2, y_train2)
clf = clf.fit(y,z)

In [ ]:
y_pred = logreg.predict(x_test)
y_pred2 = logreg2.predict(x_test2)

In [ ]:
matrix = metrics.confusion_matrix(y_test, y_pred)
matrix

In [ ]:
matrix2 = metrics.confusion_matrix(y_test2, y_pred2)
matrix2

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(classification_report(y_test2,y_pred2))

In [ ]:
import seaborn as sns

In [ ]:
#Taytay
uniHealth

In [ ]:
#Antipolo
uniHealth2

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.heatmap(pd.DataFrame(matrix), annot=True, cmap="coolwarm",fmt='g')
ax.xaxis.set_label_position("top")
ax.set_xticklabels([uniHealth[0],uniHealth[1],uniHealth[2],uniHealth[3]])
ax.set_yticklabels([uniHealth[0],uniHealth[1],uniHealth[2],uniHealth[3]])
plt.title('Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.heatmap(pd.DataFrame(matrix2), annot=True, cmap="coolwarm",fmt='g')
ax.xaxis.set_label_position("top")
ax.set_xticklabels([uniHealth2[0],uniHealth2[1],uniHealth2[2],uniHealth2[3],uniHealth2[4],uniHealth2[5]])
ax.set_yticklabels([uniHealth2[0],uniHealth2[1],uniHealth2[2],uniHealth2[3],uniHealth2[4],uniHealth2[5]])
plt.title('Matrix')
plt.show()

In [ ]:
skplt.metrics.plot_cumulative_gain(y_test, y_pred)

plt.show()

In [ ]:
skplt.metrics.plot_cumulative_gain(y_test2, y_pred2)

plt.show()